In [39]:
# Importing the Keras libraries and packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

In [40]:
from zipfile import ZipFile
file_name = "Dataset.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('finish')

finish


In [41]:
#creating model and layers
model=Sequential()
#step 1: Convolution layer
model.add(Conv2D(64,(3,3),input_shape=(64,64,3),activation='relu'))
#step2: pooling
model.add(MaxPooling2D(pool_size=(2,2)))
#step 1: Convolution layer
model.add(Conv2D(128,(3,3),activation='relu'))
#step2: pooling
model.add(MaxPooling2D(pool_size=(2,2)))
#step3: flatten
model.add(Flatten())
#full connection
model.add(Dense(128, activation='relu')) #hidden layer
model.add(Dense(1, activation='sigmoid')) #hidden layer

In [42]:
# Compiling the CNN
model.compile(optimizer ='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [43]:
#preprocessing of image
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255
                                 , shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255
                                 , shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)


In [44]:
training_set=train_datagen.flow_from_directory('Dataset/Train',
                                              target_size=(64,64),
                                              batch_size=32,
                                              class_mode='binary')


Found 574 images belonging to 2 classes.


In [45]:
test_set=train_datagen.flow_from_directory('Dataset/Test',
                                              target_size=(64,64),
                                              batch_size=32,
                                              class_mode='binary')

Found 246 images belonging to 2 classes.


In [46]:
#train the model
model.fit_generator(training_set,validation_data=test_set)

Instructions for updating:
Please use Model.fit, which supports generators.
18/18 [==============================] - 21s 1s/step - loss: 0.7685 - accuracy: 0.6132 - val_loss: 0.5360 - val_accuracy: 0.8211


In [47]:
model.save("cnn.model")

INFO:tensorflow:Assets written to: cnn.model\assets


In [48]:
import tensorflow.keras as tf
def detect(directory):
    mymodel = tf.models.load_model('cnn.model')
    import numpy as np
    from keras.preprocessing import image 
    test_image = image.load_img(directory,target_size=(64,64)) 
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image,axis=0)   ### 1-d array
    result = mymodel.predict(test_image)
    if result[0][0] == 1: #COVID-negative
        return 0
 
    else: #COVID-positive
        return 1

In [49]:
from gtts import gTTS
import playsound
import os
import time
import matplotlib.pyplot as plt
import pyttsx3
import speech_recognition as sr
import urllib.request
import re
import webbrowser
import warnings
warnings.filterwarnings('ignore')

In [50]:
###text to speech recognition 
def speak(text):
    language='en'
    engine=pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [51]:
#url='file:///C:/Users/acer/Downloads/Covid%2019%20News_%20Covid%2019%20Latest%20News%20and%20Headlines%20Today%20_%20Hindustan%20Times.html'
url='https://www.livemint.com/topic/covid-19'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
values={'s':'basics','submit':'search'}
data=urllib.parse.urlencode(values)
data=data.encode('utf-8')
req=urllib.request.Request(url,data,headers={'User-Agent': 'Mozilla/5.0'})
resp=urllib.request.urlopen(req)
info=resp.read()

In [52]:
print(info)

b'<!DOCTYPE HTML><html lang="en"><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<meta name="viewport" content="width=device-width, user-scalable=yes">\n<link rel="dns-prefetch" href="https://cdn.ampproject.org"> <link rel="dns-prefetch" href="https://images.livemint.com"> <link rel="dns-prefetch" href="https://www.livemint.com"> <link rel="dns-prefetch" href="https://www.google-analytics.com"> <link rel="dns-prefetch" href="https://fonts.googleapis.com"> <link rel="dns-prefetch" href="https://www.google.com"> <link rel="dns-prefetch" href="https://ajax.googleapis.com"> <link rel="dns-prefetch" href="https://players.brightcove.net"> <link rel="dns-prefetch" href="https://www.googletagservices.com"> <link rel="dns-prefetch" href="https://googleads.g.doubleclick.net"> <link rel="dns-prefetch" href="https://static.adsafeprotected.com"> <link rel="dns-prefetch" href="https://fonts.googleapis.com"> <link rel="dns-prefetch" href="//tpc.googlesyndication.com"> <link

In [53]:
paragraphs1=re.findall(r'<a href="/news/world/(.*?)-\d{14}',str(info))
paragraphs2=re.findall(r'<a href="/news/india/(.*?)-\d{14}',str(info))
paragraphs3=re.findall(r'<a href="/science/health/(.*?)-\d{14}',str(info))

In [54]:
paragraphs3

['germany-expects-covid-19-vaccine-early-next-year',
 '80-covid-19-patients-have-vitamin-d-deficiency-suggests-study',
 'oxford-covid-vaccine-astrazeneca-expects-vaccine-data-this-year',
 'can-vitamin-d-help-fight-covid-19']

In [59]:
while True:
    src=sr.Recognizer()
    r=sr.Recognizer()
    print("Hey There...I am Listening")
    with sr.Microphone() as source:
        audio=src.listen(source)
        query=r.recognize_google(audio, language='en-US')
        print(query)
    if 'hi Alexa' in query:
        print('Hey buddy!How can I help you?')
        speak('Hey buddy!How can I help you?')
    elif 'what can you do' in query:
        print('I am Alexa,your assistant!I can do perform various tasks.What do you want me to do?')
        speak('I am Alexa,your assistant!I can do perform various tasks.What do you want me to do?')
    elif 'detect' in query:
        print('Enter the directory of the X-ray image of the lungs')
        speak('Enter the directory of the X-ray image of the lungs')
        #ip=r'C:\Users\acer\Desktop\Udemy\ML_Projects\sample_project\Dataset\Test\Covid\24.jpeg'
        #ip=r'C:\Users\acer\Desktop\Udemy\ML_Projects\sample_project\Dataset\Test\Non Covid\164.jpeg'
        ip=input("")
        print(ip)
        detection=detect(ip)
        if detection==0:
            print('I am glad that this patient is detected covid negative, but should continue to follow the necessary precautions')
            speak('I am glad that this patient is detected covid negative, but should continue to follow the necessary precautions')
        else:
            print('Alas! This patient is covid positive but on the bright side the current recovery rate id 92.2%')
            speak('Alas! This patient is covid positive but on the bright side the current recovery rate id 92.2%')
    elif 'precautions' in query:
        print('If COVID-19 is spreading in your community, stay safe by taking some simple precautions, such as physical distancing, wearing a mask, keeping rooms well ventilated, avoiding crowds, cleaning your hands, and coughing into a bent elbow or tissue. Check local advice where you live and work. Do it all!')
        speak('If COVID-19 is spreading in your community, stay safe by taking some simple precautions, such as physical distancing, wearing a mask, keeping rooms well ventilated, avoiding crowds, cleaning your hands, and coughing into a bent elbow or tissue. Check local advice where you live and work. Do it all!')
        
    elif 'news' in query:
        print(paragraphs1)
        speak(paragraphs1)
        print(paragraphs2)
        speak(paragraphs2)
        print(paragraphs3)
        speak(paragraphs3)
        
    
    elif 'done' in query:
        print('Bye bye! have a great day ahead!')
        speak('Bye bye! have a great day ahead!')
        break
        
    else:
        print('I am sorry,I did not understand')
        speak('I am sorry,I did not understand')

Hey There...I am Listening
hi Alexa
Hey buddy!How can I help you?
Hey There...I am Listening
what can you do
I am Alexa,your assistant!I can do perform various tasks.What do you want me to do?
Hey There...I am Listening
can you detect if a person has covid aur not
Enter the directory of the X-ray image of the lungs
C:\Users\acer\Desktop\Udemy\ML_Projects\sample_project\Dataset\Test\Covid\24.jpeg
C:\Users\acer\Desktop\Udemy\ML_Projects\sample_project\Dataset\Test\Covid\24.jpeg
Alas! This patient is covid positive but on the bright side the current recovery rate id 92.2%
Hey There...I am Listening
can you detect a person has covid or not
Enter the directory of the X-ray image of the lungs
C:\Users\acer\Desktop\Udemy\ML_Projects\sample_project\Dataset\Test\Non Covid\164.jpeg
C:\Users\acer\Desktop\Udemy\ML_Projects\sample_project\Dataset\Test\Non Covid\164.jpeg
I am glad that this patient is detected covid negative, but should continue to follow the necessary precautions
Hey There...I am L

In [60]:
speak('Thank You!!')